# Exploration des données pour conseiller nos clients sur l'achat de nos véhicules disponibles sur notre plateforme AutoPredict


Vous pourrez, ci-dessous, consulter nos statistiques pour savoir quels types de véhicules nous pourrions vous orienter en fonction de vos besoins.

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import numpy as np

# Activation du mode notebook (important pour VS Code)
pyo.init_notebook_mode(connected=True)

# Chargement du fichier CSV
df = pd.read_csv("voiture.csv")

print(len(df))

df.head()

# Data cleaning

df.columns = df.columns.str.lower().str.replace(" ", "_")
df.rename(columns={
    'engine_fuel_type': 'fuel_type',
    'engine_hp': 'hp',
    'engine_cylinders': 'cylinders',
    'transmission_type': 'transmission',
    'driven_wheels': 'drive',
    'number_of_doors': 'doors',
    'market_category': 'market',
    'vehicle_size': 'size',
    'vehicle_style': 'style',
    'msrp': 'price'
}, inplace=True)

print('Number of duplicates are:', df.duplicated().sum())
df = df.drop_duplicates()

print('Number of missing values in each column:')
print(df.isnull().sum())


# Handle missing values
df['fuel_type'] = df['fuel_type'].fillna('regular unleaded')
df['hp'] = df['hp'].fillna(0)
df['cylinders'] = df['cylinders'].fillna(0)
df['doors'] = df['doors'].fillna(df['doors'].mean())

# Remove rows with unknown transmission
df = df[df['transmission'] != 'UNKNOWN']

print(len(df))  # Final dataset size after cleaning

# Nettoyage : colonnes nécessaires et valeurs manquantes
df_clean = df[['price', 'transmission']].dropna()

# Suppression des transmissions UNKNOWN
df_clean = df_clean[df_clean['transmission'] != 'UNKNOWN']

# Création de quantiles de prix
df_clean['price Quantile'] = pd.qcut(df_clean['price'], q=5)

# Conversion en string pour Plotly
df_clean['price Quantile'] = df_clean['price Quantile'].astype(str)

# Tri explicite des catégories dans l'ordre croissant
ordered_quantiles = sorted(df_clean['price Quantile'].unique(), key=lambda x: float(x.split(',')[0][1:]))
df_clean['price Quantile'] = pd.Categorical(df_clean['price Quantile'], categories=ordered_quantiles, ordered=True)

# Regroupement
grouped = df_clean.groupby(['price Quantile', 'transmission']).size().reset_index(name='Nombre de véhicules')

# Création du graphique
fig = px.bar(
    grouped,
    x='price Quantile',
    y='Nombre de véhicules',
    color='transmission',
    text='Nombre de véhicules',
    title="Répartition des types de transmissions de véhicule par quantile de prix",
    labels={'price Quantile': 'Catégorie de prix (quantiles price)'}
)

# Mise en forme
fig.update_layout(
    xaxis_tickangle=-45,
    barmode='stack',
    legend_title="Type de transmission",
    yaxis_title="Nombre de véhicules",
    font=dict(size=14)
)

fig.update_traces(textposition='inside')

# Affichage dans le notebook
pyo.iplot(fig)


11914
Number of duplicates are: 715
Number of missing values in each column:
make               0
model              0
year               0
fuel_type          3
hp                69
cylinders         30
transmission       0
drive              0
doors              6
market          3376
size               0
style              0
highway_mpg        0
city_mpg           0
popularity         0
price              0
dtype: int64
Deleted outliers are: 1403
9784


C:\Users\yannr\AppData\Local\Temp\ipykernel_24536\1602079606.py:81: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

### Analyse croisée de la répartition des types de transmission en fonction du prix et du style de véhicule

#### Répartition des types de transmission selon les quantiles de prix

Le premier graphique présente la distribution des types de transmission (automatique, manuelle, automatisée manuelle, etc.) au sein de cinq quantiles de prix (MSRP). Une tendance nette se dégage :

- **Les véhicules à transmission automatique** dominent largement dans les **quantiles intermédiaires à élevés**, représentant souvent **plus de 80 %** des véhicules de ces catégories. Ce type de transmission est typique des véhicules destinés à un usage confortable ou urbain.
- **Les transmissions manuelles** sont **majoritairement présentes dans les véhicules les moins chers** (1er quantile), puis décroissent fortement à mesure que le prix augmente.
- Les **transmissions automatisées manuelles** apparaissent de manière marginale, surtout dans les véhicules haut de gamme.

Ces observations traduisent une corrélation directe entre le **niveau de prix** d’un véhicule et son **type de transmission**.

In [27]:
# Préparation des données
df_clean = df[['transmission', 'style']].dropna()
df_clean = df_clean[df_clean['transmission'] != 'UNKNOWN']

# Agrégation par transmission et style
style_trans = df_clean.groupby(['transmission', 'style']).size().reset_index(name='Nombre de véhicules')

# Création du graphique sunburst
fig = px.sunburst(
    style_trans,
    path=['transmission', 'style'],
    values='Nombre de véhicules',
    title="Répartition des styles de véhicule selon le type de transmission"
)

# Affichage du pourcentage et étiquettes
fig.update_traces(
    textinfo='label+percent entry',
    insidetextorientation='radial'
)

# Taille du graphique
fig.update_layout(
    font=dict(size=14),
    width=900,
    height=700
)

# Affichage dans VS Code
pyo.iplot(fig)


#### Répartition des styles de véhicule selon le type de transmission

Le second graphique, un diagramme sunburst, permet de visualiser la répartition des **styles de véhicule** en fonction du **type de transmission** :

- Les **transmissions automatiques** concernent principalement des véhicules de type **SUV** et **berlines (Sedan)**, c’est-à-dire des modèles généralement plus spacieux, orientés vers le confort et souvent utilisés dans des contextes urbains ou familiaux.
- Les **transmissions manuelles** sont davantage associées à des **coupés**, **hatchbacks**, ou encore des **pickups**. Ces styles sont souvent liés à des usages plus économiques, sportifs ou professionnels.
- Les transmissions automatisées et directes (DIRECT_DRIVE) sont très minoritaires, ce qui les rend peu représentatives pour l’analyse.

#### Corrélation entre les deux graphiques

Les deux visualisations convergent vers une **relation structurelle entre le prix, le type de transmission et le style du véhicule** :

| Type de transmission | Styles dominants               | Gamme de prix associée  |
|----------------------|--------------------------------|-------------------------|
| Automatique          | SUV, Sedan                     | Moyenne à élevée        |
| Manuelle             | Coupé, Hatchback, Pickup       | Basse à moyenne         |
| Semi-automatique     | Divers véhicules haut de gamme | Élevée                  |

Cette corrélation permet de tirer des conclusions utiles à la fois en matière de compréhension du marché et dans une logique de conseil à l’achat.

#### En tant que vendeur:

Dans une optique de vente ou de recommandation:

- Pour un **client recherchant le confort ou un usage familial**, les véhicules **automatiques de type SUV ou berline** seront les plus adaptés.
- Pour un **client au budget restreint**, un **véhicule manuel** de type **compact** ou **coupé** sera plus accessible.
- Pour un **profil professionnel ou rural**, le **pickup manuel** reste un bon compromis entre robustesse et maîtrise du budget.

On peut **établir un profil type des acheteurs selon leur budget, leur usage et leurs attentes en matière de confort ou performance.**


In [28]:
# Suppression des lignes avec valeurs manquantes
df_clean = df.dropna(subset=['hp', 'highway_mpg', 'city_mpg', 'cylinders']).copy()

# Calcul de la consommation moyenne
df_clean['Average MPG'] = (df_clean['highway_mpg'] + df_clean['city_mpg']) / 2

# Filtrage : suppression des moteurs à 0 cylindre (valeurs aberrantes)
df_clean = df_clean[df_clean['cylinders'] > 0]

# Tranches de puissance moteur
df_clean['HP Range'] = pd.qcut(df_clean['hp'], q=5, labels=[
    'Très faible', 'Faible', 'Moyenne', 'Élevée', 'Très élevée'])

# Conversion des cylindres à entier
df_clean['cylinders'] = df_clean['cylinders'].round().astype('Int64')

# Agrégation
df_filtered = df_clean.groupby(['HP Range', 'cylinders']).agg(
    avg_mpg=('Average MPG', 'mean'),
    std_mpg=('Average MPG', 'std')
).reset_index()

# Graphique interactif
fig = px.line(
    df_filtered,
    x='HP Range',
    y='avg_mpg',
    error_y='std_mpg',
    color='cylinders',
    markers=True,
    title="Consommation moyenne par puissance moteur",
    labels={
        "HP Range": "Type de puissance moteur",
        "avg_mpg": "Consommation moyenne (MPG)",
        "cylinders": "Nombre de cylindres"
    }
)

fig.update_layout(
    legend_title_text='Nb Cylindres',
    yaxis_title="Consommation moyenne (MPG)",
    xaxis_title="Type de puissance moteur"
)

fig.show()


C:\Users\jerem\AppData\Local\Temp\ipykernel_24452\1954858973.py:18: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




## Analyse croisée de la consommation moyenne selon la puissance moteur et le nombre de cylindres

Ce graphique interactif illustre la consommation moyenne de carburant (exprimée en miles par gallon – MPG) en fonction de deux caractéristiques mécaniques clés : la puissance du moteur et le nombre de cylindres. L’objectif est de mettre en évidence les liens entre performance moteur et rendement énergétique.

### Distribution générale : plus de puissance, moins de sobriété

On observe une **diminution progressive de la consommation moyenne (MPG) à mesure que la puissance augmente**. Cela confirme l’intuition selon laquelle les véhicules les plus puissants, quelle que soit leur configuration mécanique, sont aussi les moins économes en carburant. Cette tendance peut s’expliquer par plusieurs facteurs :

- Une **augmentation de la cylindrée et du couple** nécessite plus d’énergie, donc plus de carburant.
- Les véhicules puissants sont souvent plus **lourds, rapides ou orientés performance**, ce qui augmente naturellement la consommation.
- Une **sollicitation plus importante du moteur** dans les plages de puissance supérieures induit des pertes énergétiques.

### L'influence du nombre de cylindres

En croisant la puissance avec le nombre de cylindres, plusieurs observations se dégagent :

- Les **moteurs 3 et 4 cylindres** restent globalement les plus sobres, même dans les tranches moyennes de puissance.
- À partir de **6 cylindres**, la consommation moyenne augmente sensiblement et suit une évolution relativement stable selon la puissance.
- Les moteurs à **10, 12, voire 16 cylindres** présentent une consommation très élevée, cohérente avec leur usage dans des véhicules de luxe, sportifs ou utilitaires haut de gamme.

Cette répartition démontre que le **nombre de cylindres est un indicateur fiable de rendement énergétique**, en complément de la puissance brute du moteur.

### Hypothèses explicatives et interprétations

Plusieurs hypothèses permettent de compléter l’interprétation de ces résultats :

- **Optimisation technologique** : certains moteurs 4 cylindres très performants affichent une consommation contenue grâce aux innovations (turbo, hybridation, désactivation de cylindres, etc.).
- **Catégories des véhicules** : les véhicules à fort nombre de cylindres appartiennent souvent à des catégories plus énergivores (SUV, sportives, utilitaires).
- **Évolution réglementaire** : des motorisations avec une réduction de cylindrée apparaissent dans les tranches de puissance élevées, pour concilier performance et sobriété.

### Conclusion

Ce graphique montre que la consommation moyenne des véhicules est **étroitement corrélée à la puissance moteur et au nombre de cylindres**. Si la tendance générale confirme qu’un moteur plus puissant consomme davantage, des nuances apparaissent selon les technologies utilisées et les segments de véhicules. Cette analyse permet d'évaluer les compromis entre performance et efficacité énergétique pour les voitures que nous souhaitons vendre. Il serait interessant de faire une étude comparative des prix des véhicules en fonction de son moteur et d'observer quels sont les catégories de voitures qui se démarquent.



In [29]:
# Nettoyage et préparation
df_clean = df.dropna(subset=['hp', 'market', 'price']).copy()
df_clean['HP Range'] = pd.qcut(df_clean['hp'], q=5, labels=[
    'Très faible', 'Faible', 'Moyenne', 'Élevée', 'Très élevée'])
df_clean['market'] = df_clean['market'].str.split(',')
df_exploded = df_clean.explode('market')

# Calcul du prix moyen
df_price = df_exploded.groupby(['HP Range', 'market'])['price'].mean().reset_index()
df_price['price'] = df_price['price'].round(0)

# Nombre de véhicules
vehicle_counts = df_exploded.groupby(['HP Range', 'market']).size().reset_index(name='Nombre de véhicules')
df_price_annotated = df_price.merge(vehicle_counts, on=['HP Range', 'market'])

# Création du graphique avec annotations visibles
fig = go.Figure()
categories = df_price_annotated['market'].unique()
hp_order = ['Très faible', 'Faible', 'Moyenne', 'Élevée', 'Très élevée']

for category in categories:
    df_cat = df_price_annotated[df_price_annotated['market'] == category]
    fig.add_trace(go.Bar(
        x=df_cat['HP Range'],
        y=df_cat['price'],
        name=category,
        text=df_cat['Nombre de véhicules'],
        textposition='outside',  # texte visible au-dessus des barres
        hovertemplate='<b>%{x}</b><br>Catégorie: ' + category + '<br>Prix moyen: $%{y}<br>Nombre de véhicules: %{text}',
    ))

# Mise en forme
fig.update_layout(
    barmode='group',
    title="Prix moyen (price) par catégorie de voiture et puissance moteur (avec nombre de véhicules)",
    xaxis_title="Type de puissance moteur",
    yaxis_title="Prix moyen des véhicules ($)",
    legend_title="Catégorie de véhicule",
    xaxis={'categoryorder': 'array', 'categoryarray': hp_order}
)

fig.show()



C:\Users\jerem\AppData\Local\Temp\ipykernel_24452\838335851.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\jerem\AppData\Local\Temp\ipykernel_24452\838335851.py:13: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Analyse croisée : prix moyen des véhicules par catégorie de voitures et puissance moteur

Le graphique représente le **prix moyen (`price`)** des véhicules selon leur **catégorie de voitures** et leur **Type de puissance moteur**, tout en indiquant le **nombre de véhicules** disponibles dans chaque groupe. Cette visualisation vise à évaluer dans quelle mesure il est possible pour un client d’opter pour un véhicule **puissant** tout en maîtrisant son **budget**, malgré une consommation potentiellement plus élevée.

### Plus de puissance veut dire plus cher ?

Une tendance générale se dégage : **le prix augmente avec la puissance moteur**. Toutefois, cette relation n’est **pas systématique** :

- La catégorie **"Luxury"** s’étend sur toutes les tranches de puissance, avec des prix relativement **stables** autour de 60 000 $ à 90 000 $.
- Les catégories **"High-Performance"** et **"Exotic"** affichent des **prix très élevés** dès que la puissance atteint les tranches "Élevée" et "Très élevée", culminant jusqu’à 250 000 $ pour les véhicules "Exotic".
- Certaines catégories comme **"Crossover"** ou **"Performance"** présentent une **croissance plus modérée** du prix malgré l’augmentation de la puissance.

### Des compromis accessibles

Certaines combinaisons permettent au client de viser une **voiture puissante à un budget modéré** :

- Les véhicules **"Performance"**, présents dans les tranches "Élevée" et "Très élevée", affichent un **prix moyen de 40 000 à 60 000 $**, soit bien en dessous des véhicules "Luxury" ou "Exotic" de puissance équivalente.
- La catégorie **"Crossover"**, bien que typiquement associée à des usages utilitaires, reste présente dans toutes les tranches de puissance, avec un **prix relativement stable (~40 000 $)**.

Ces segments offrent des **alternatives intéressantes** pour un acheteur souhaitant privilégier la puissance sans entrer dans les sphères du très haut de gamme.

### Hypothèse de rentabilité pour faire une concession sur la consommation

Il peut être **pertinent** de faire une **concession sur la consommation**, si le **prix d’acquisition reste abordable**. En effet :

- Les véhicules **puissants mais abordables** en prix moyen comme certains "Performance" ou "Flex Fuel" peuvent être **rentables à long terme**, surtout si la consommation est compensée par un usage limité et maîtrisé. A **court terme", si l'utilisation du véhicule est de courtes durées.
- À l’inverse, les véhicules très puissants dans les catégories **"Exotic"** ou **"High-Performance"**, en plus d’une consommation élevée, exigent un **investissement initial considérable**, réduisant fortement leur rentabilité dans une perspective classique.

### Équilibre entre performance et budget

Ce graphique met en évidence un **équilibre atteignable** entre puissance moteur et prix d’achat pour certaines catégories :

- **"Performance"**, **"Crossover"** ou même certains véhicules **"Luxury"** de gamme intermédiaire offrent des compromis intéressants.
- Ces segments permettent d’accéder à des véhicules **puissants mais non excessifs en prix**, pour un public recherchant performance et budget maîtrisé.

### Conclusion

> Il est possible pour un client d’acheter une **voiture puissante** en acceptant une **consommation plus élevée**, **à condition de bien choisir la catégorie de véhicule**.  
> L’analyse montre que certaines **catégories de marché offrent une puissance élevée à un prix modéré**, ouvrant la voie à une rentabilité acceptable malgré la consommation.

Ainsi, l’acheteur rationnel dispose d’une **marge de manœuvre** pour arbitrer entre puissance, consommation et coût, en fonction de ses priorités comme la performance, budget, confort ou image.

---
# Diversité des gammes constructeurs et impact sur la popularité : analyse des modèles, segments et positionnements de marques

In [10]:
# Suppression des doublons et des lignes avec valeurs manquantes
df_clean1 = df.dropna(subset=['make', 'model']).copy()
df_clean1 = df_clean1.drop_duplicates(subset=['make', 'model'])

# Comptage des modèles distincts par marque
model_counts = df_clean1.groupby('make')['model'].nunique().reset_index()
model_counts.columns = ['Marque', 'Nombre de modèles']
model_counts = model_counts.sort_values(by='Nombre de modèles', ascending=False)

# Création du bar chart avec une couleur douce et harmonieuse
fig = px.bar(
    model_counts.head(15),
    x='Marque',
    y='Nombre de modèles',
    text='Nombre de modèles',
    title="Top 15 des marques avec le plus grand nombre de modèles distincts",
    color_discrete_sequence=["#4682B4"]  # bleu acier doux
)

# Mise en forme visuelle
fig.update_layout(
    title=dict(x=0.5, font=dict(size=20)),
    font=dict(size=14),
    xaxis=dict(
        title=dict(text="Marque", font=dict(size=16)),
        tickfont=dict(size=13),
        tickangle=-30
    ),
    yaxis=dict(
        title=dict(text="Nombre de modèles", font=dict(size=16)),
        tickfont=dict(size=13),
        gridcolor='lightgrey'
    ),
    plot_bgcolor='white',
    margin=dict(t=80, b=60, l=60, r=40)
)

fig.update_traces(
    textposition="outside",
    marker=dict(line=dict(width=1, color='black'))
)

fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

###  Analyse du graphique – Nombre de modèles distincts par marque

Ce graphique met en évidence les marques proposant le plus grand nombre de modèles distincts dans la base de données AutoPredict. L’objectif est d’identifier les constructeurs ayant une offre étendue, ce qui peut influencer la recommandation de véhicules pour les clients.
  

####  Observations principales

- **Chevrolet**, **Ford** et **Lexus** dominent ce classement avec respectivement 63, 51 et 46 modèles distincts.

- On retrouve ensuite des marques comme **Mercedes-Benz**, **BMW**, **Toyota** ou **Infiniti**, avec 38 à 40 modèles.

- En bas du classement, **Volvo**, **Cadillac** ou **GMC** comptent autour de 26 à 31 modèles.



#### Interprétations

- Les marques en haut du classement adoptent généralement une stratégie généraliste : elles visent plusieurs types de clients avec des gammes variées (citadine, SUV, berline, utilitaire...).
- Les marques avec moins de modèles ont souvent un positionnement plus ciblé ou haut de gamme.
- Certaines marques comme **Lexus**, bien qu’étant premium, proposent tout de même une gamme étendue pour couvrir plusieurs segments premium.



####  Implications pour AutoPredict

- Une marque avec un large choix de modèles est plus susceptible d’être recommandée**, car elle peut répondre à différents critères utilisateurs (prix, puissance, style...).
- À l'inverse, une marque avec peu de modèles peut tout de même être mise en avant pour des besoins spécifiques (ex : SUV de luxe, sécurité maximale, etc.).



In [11]:
# Top 5 marques les plus présentes
top_5_makes = model_counts.head(5)['Make'].tolist()

# Nettoyage
df_clean2 = df.dropna(subset=['Make', 'Model', 'Market Category']).copy()
df_clean2 = df_clean2.drop_duplicates(subset=['Make', 'Model', 'Market Category'])

# Explosion des segments multiples
df_clean2['Market Category'] = df_clean2['Market Category'].str.split(',')
df_exploded2 = df_clean2.explode('Market Category')
df_exploded2 = df_exploded2[df_exploded2['Make'].isin(top_5_makes)]

fig = px.bar(
    segment_counts,
    x='Make',
    y='Nb modèles',
    color='Market Category',
    text='Nb modèles',
    title="Répartition des segments pour les 5 marques principales",
    color_discrete_sequence=px.colors.qualitative.D3  # Palette plus contrastée
)

fig.update_layout(
    barmode='stack',
    title=dict(x=0.5, font=dict(size=20)),
    font=dict(size=14),
    xaxis=dict(
        title="Marque",
        tickangle=0,  # Écriture horizontale
        titlefont=dict(size=16),
        tickfont=dict(size=13, family='Arial', color='black'),  # Tu peux ajouter font='bold' avec fig.add_annotation si besoin de plus fort gras
    ),
    yaxis=dict(
        title="Nombre de modèles",
        titlefont=dict(size=16),
        tickfont=dict(size=13),
        gridcolor='lightgrey'
    ),
    legend_title_text='Segment (Market Category)',
    plot_bgcolor='white',
    margin=dict(t=80, b=60, l=60, r=40)
)

# Bordures plus fines et lisibles
fig.update_traces(
    textposition='inside',
    marker=dict(line=dict(width=0.5, color='black'))
)

fig.show()



KeyError: 'Make'

###  Analyse du graphique – Répartition des segments pour les 5 marques principales

Ce graphique illustre comment les 5 marques ayant le plus grand nombre de modèles répartissent leur offre entre les différents segments de marché (ou "Market Categories").  
L’objectif est d’évaluer si ces marques ont une stratégie diversifiée et homogène, ou si elles se concentrent sur des segments spécifiques, révélant une spécialisation.



####  Observations principales

- **BMW** et **Mercedes-Benz** se distinguent par une très forte concentration de modèles dans le segment **"Luxury"** :
  - **BMW** : 70 modèles "Luxury"
  - **Mercedes-Benz** : 83 modèles "Luxury"
- **Mercedes-Benz** est la marque la plus segmentée avec des modèles dans presque toutes les catégories : Crossover, Performance, Factory Tuner, etc.
- **Chevrolet** et **Ford** présentent une répartition plus équilibrée entre les segments, bien que moins dense en nombre total de modèles.
- **Lexus** affiche une structure proche de Mercedes-Benz mais dans une échelle plus réduite : fort sur le "Luxury", un peu de Performance, Hybrid, Crossover.


####  Points clés à retenir

- **BMW** et **Mercedes-Benz** concentrent l’essentiel de leur offre sur le segment **"Luxury"** (respectivement 70 et 83 modèles), en cohérence avec leur positionnement haut de gamme.
- **Mercedes-Benz** se démarque en couvrant presque **tous les segments existants**, révélant une **stratégie à la fois premium et étendue**.
- **Ford** et **Chevrolet** adoptent une stratégie plus **généraliste**, avec une répartition équilibrée sur plusieurs segments (Crossover, Performance, Hybrid...).
- **Lexus** suit une structure proche de **Mercedes-Benz**, mais avec un volume plus modeste.

---

#### Ce que cela implique pour AutoPredict

- Les marques aux offres **diversifiées** peuvent répondre à **plusieurs profils clients** : elles sont donc **pertinentes pour les recommandations personnalisées**.
- À l’inverse, les marques très **spécialisées** conviendront mieux à des utilisateurs avec des attentes ciblées.
- Comprendre cette répartition permet d'**affiner notre moteur de suggestion**, en croisant ces informations avec d’autres critères (prix, performance, consommation...).


In [ ]:
# Nettoyage
df_clean3 = df.dropna(subset=['Make', 'Model', 'Market Category', 'Popularity']).copy()
df_clean3 = df_clean3.drop_duplicates(subset=['Make', 'Model', 'Market Category'])

# Explosion des catégories
df_clean3['Market Category'] = df_clean3['Market Category'].str.split(',')
df_exploded3 = df_clean3.explode('Market Category')

# Diversité de segments
segment_diversity = df_exploded3.groupby('Make')['Market Category'].nunique().reset_index()
segment_diversity.columns = ['Make', 'Diversité Segment']

# Fusion avec popularité
popularity_data = df_exploded3.groupby('Make')['Popularity'].mean().reset_index()
popularity_data.columns = ['Make', 'Popularité Moyenne']

# Dataset final
merged_df = pd.merge(segment_diversity, popularity_data, on='Make')

# Graphe en boite
fig = px.box(
    merged_df,
    x='Diversité Segment',
    y='Popularité Moyenne',
    color='Diversité Segment',
    title="Distribution de la popularité moyenne selon la diversité des segments",
    labels={
        'Diversité Segment': "Nombre de segments distincts",
        'Popularité Moyenne': "Popularité moyenne"
    }
)

fig.update_layout(
    title=dict(x=0.5, font=dict(size=20)),
    font=dict(size=14),
    xaxis=dict(
        title="Diversité des segments",
        tickmode='linear',
        titlefont=dict(size=16),
        tickfont=dict(size=13)
    ),
    yaxis=dict(
        title="Popularité moyenne",
        titlefont=dict(size=16),
        tickfont=dict(size=13),
        gridcolor='lightgrey'
    ),
    showlegend=False,
    plot_bgcolor='white',
    margin=dict(t=80, b=40, l=60, r=40)
)

fig.update_traces(marker=dict(line=dict(width=1.5)), selector=dict(type='box'))

fig.show()

### Analyse du graphique – Popularité moyenne selon la diversité des segments couverts

Ce graphique en boîte illustre la relation entre la diversité des segments couverts par une marque (axe X) et sa popularité moyenne (axe Y).  
Chaque boîte représente la distribution de la popularité des marques ayant un même nombre de segments distincts dans leur catalogue.


#### Points clés à retenir

- Les marques avec une faible diversité (1 à 3 segments) ont généralement une popularité faible à modérée, avec peu d’écart entre elles.
- À partir de 6 segments, on observe une montée en popularité moyenne, mais surtout une variabilité plus importante.
- Les marques couvrant 8 à 9 segments présentent les niveaux de popularité les plus élevés, ainsi qu’une dispersion marquée, suggérant des positions très différenciées sur le marché.


#### Interprétation

- Une diversité élevée en segments semble être un facteur associé à une plus grande visibilité ou attractivité des marques.
- Cela pourrait refléter leur capacité à répondre à une demande plus large, ce qui les rend plus populaires sur le marché.
- Toutefois, la grande dispersion sur certains niveaux de diversité indique que la popularité ne dépend pas uniquement de la variété, mais aussi d’autres leviers comme l’image de marque, la stratégie marketing ou les prix.


#### Conclusion globale de l’analyse

Au cours de cette analyse, nous avons étudié :
- Quelles marques proposent le plus de modèles (ex. **Chevrolet**, **Ford**, **Lexus**)
- Comment ces modèles sont répartis entre les segments (ex. **BMW**, **Mercedes-Benz**, **Lexus** très concentrées sur le luxe, **Ford** et **Chevrolet** plus généralistes)
- Et comment la diversité de l’offre influence la popularité

Ces résultats montrent que les marques les plus diversifiées sont généralement mieux positionnées pour toucher une audience large, et gagnent en popularité.  
Cependant, certaines marques plus spécialisées, comme **Lexus** ou **Mercedes-Benz**, conservent une forte notoriété grâce à leur positionnement haut de gamme.

Cela confirme l’intérêt d’intégrer la diversité de l’offre comme critère dans notre moteur de recommandation AutoPredict, afin de mieux cibler les profils utilisateurs et proposer des véhicules réellement adaptés à leurs attentes.


In [30]:
# Analyse de la dépréciation moyenne par année

# On garde uniquement les voitures dont l'année et le prix sont valides
df_year_price = df[['year', 'price']].dropna()
df_year_price = df_year_price[df_year_price['year'] > 1990]
df_year_price = df_year_price[df_year_price['price'] > 1000]

# Calcul du prix moyen par année
avg_price_by_year = df_year_price.groupby('year')['price'].agg(['mean', 'count']).reset_index()

# Tracé du graphique
fig = px.line(
    avg_price_by_year,
    x='year',
    y='mean',
    markers=True,
    text=avg_price_by_year['count'],
    labels={'year': 'Année du véhicule', 'mean': 'Prix moyen (USD)'},
    title='Dépréciation moyenne des véhicules par année'
)

fig.update_traces(
    hovertemplate='Année: %{x}<br>Prix moyen: $%{y:.2f}<br>Nb véhicules: %{text}<extra></extra>',
    marker=dict(size=6)
)

fig.update_layout(
    yaxis_tickprefix='$',
    yaxis_tickformat=',.0f',
    xaxis_tickangle=-45,
    height=500,
    margin=dict(t=60, b=100),
    hovermode="closest"
)

fig.show()


In [31]:
# Analyse de la dépréciation par style de véhicule

# Garder uniquement les colonnes nécessaires et nettoyer les données
df_style_year = df[['year', 'price', 'style']].dropna()
df_style_year = df_style_year[df_style_year['year'] > 1990]
df_style_year = df_style_year[df_style_year['price'] > 1000]

# Calcul du prix moyen par style et par année
style_group = df_style_year.groupby(['style', 'year'])['price'].agg(['mean', 'count']).reset_index()

# Tracé
fig = px.line(
    style_group,
    x='year',
    y='mean',
    color='style',
    markers=True,
    hover_data={'count': True},
    labels={
        'year': 'Année du véhicule',
        'mean': 'Prix moyen (USD)',
        'style': 'Style de véhicule',
        'count': 'Nb véhicules'
    },
    title='Dépréciation par style de véhicule'
)

fig.update_traces(
    hovertemplate='Style: %{legendgroup}<br>Année: %{x}<br>Prix moyen: $%{y:.2f}<br>Nb véhicules: %{customdata[0]}<extra></extra>',
    marker=dict(size=5)
)

fig.update_layout(
    yaxis_tickprefix='$',
    yaxis_tickformat=',.0f',
    xaxis_tickangle=-45,
    height=600,
    legend_title_text='Style de véhicule',
    margin=dict(t=60, b=120),
    hovermode="closest"
)

fig.show()


In [32]:
# Analyse de la dépréciation par catégorie de marché

# Certaines voitures ont plusieurs catégories séparées par des virgules → on les éclate
df_category = df[['year', 'price', 'market']].dropna()
df_category = df_category[df_category['year'] > 1990]
df_category = df_category[df_category['price'] > 1000]

# Séparer les catégories multiples et réassembler le DataFrame
df_exploded = df_category.assign(
    market=df_category['market'].str.split(',')
).explode('market')

# Nettoyer les noms de catégories
df_exploded['market'] = df_exploded['market'].str.strip()

# Grouper par catégorie + année
cat_group = df_exploded.groupby(['market', 'year'])['price'].agg(['mean', 'count']).reset_index()

# Tracé
fig = px.line(
    cat_group,
    x='year',
    y='mean',
    color='market',
    markers=True,
    hover_data={'count': True},
    labels={
        'year': 'Année du véhicule',
        'mean': 'Prix moyen (USD)',
        'market': 'Catégorie de marché',
        'count': 'Nb véhicules'
    },
    title='Dépréciation par catégorie de marché'
)

fig.update_traces(
    hovertemplate='Catégorie: %{legendgroup}<br>Année: %{x}<br>Prix moyen: $%{y:.2f}<br>Nb véhicules: %{customdata[0]}<extra></extra>',
    marker=dict(size=5)
)

fig.update_layout(
    yaxis_tickprefix='$',
    yaxis_tickformat=',.0f',
    xaxis_tickangle=-45,
    height=600,
    legend_title_text='Catégorie marketing',
    margin=dict(t=60, b=120),
    hovermode="closest"
)

fig.show()


In [ ]:
# Suppression des doublons et des lignes avec valeurs manquantes
df_clean1 = df.dropna(subset=['Make', 'Model']).copy()
df_clean1 = df_clean1.drop_duplicates(subset=['Make', 'Model'])

# Comptage des modèles par marque
model_counts = df_clean1.groupby('Make')['Model'].count().reset_index()
model_counts.columns = ['Make', 'Nombre de modèles']
model_counts = model_counts.sort_values(by='Nombre de modèles', ascending=False)

# Création du bar chart avec une seule couleur douce
fig = px.bar(
    model_counts.head(15),
    x='Make',
    y='Nombre de modèles',
    text='Nombre de modèles',
    title=" Top 15 des marques avec le plus grand nombre de modèles distincts",
    color_discrete_sequence=["#4682B4"]  # Bleu acier doux
)

# Mise en forme visuelle
fig.update_layout(
    title=dict(x=0.5, font=dict(size=20)),  # Titre centré et plus grand
    font=dict(size=14),
    xaxis=dict(
        title="Marque",
        titlefont=dict(size=16),
        tickfont=dict(size=13),
        tickangle=-30
    ),
    yaxis=dict(
        title="Nombre de modèles",
        titlefont=dict(size=16),
        tickfont=dict(size=13),
        gridcolor='lightgrey'
    ),
    plot_bgcolor='white',
    margin=dict(t=80, b=60, l=60, r=40)
)

fig.update_traces(
    textposition="outside",
    marker=dict(line=dict(width=1, color='black'))  # Bordure légère
)

fig.show()
